In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# tsv --> read_table
train = pd.read_table('/kaggle/input/sentiment-analysis-on-movie-reviews/train.tsv.zip')
train

In [ ]:
# 다중 분류대회(감성분석)
train['Sentiment'].value_counts()
# 0 --> 매우 부정 ~ 4 --> 매우 긍정

In [ ]:
test = pd.read_table('/kaggle/input/sentiment-analysis-on-movie-reviews/test.tsv.zip')
test

In [ ]:
# 단어 등장 빈도수를 체크해서 부정/긍정 판단
# 텍스트 분류 기법의 기본
all_data = pd.concat([train,test],ignore_index=True)
all_data

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cv = CountVectorizer(max_features=200)
tfi = TfidfVectorizer()
#text = cv.fit_transform(all_data['Phrase'])
text = tfi.fit_transform(all_data['Phrase'])
text
# 17729 --> 우리 데이터셋의 단어종류
# max_features=200 해줘서 제한


In [ ]:
# 단어 200개만 추출하는 방식말고 차원축소로 접근
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=20)
text_svd = svd.fit_transform(text)
text_svd.shape

In [ ]:
#pd.DataFrame(text.todense())

In [ ]:
train_2 = text_svd[:len(train)]
test_2 = text_svd[len(train):]

In [ ]:
'''
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_jobs=-1,random_state=42)
rfc.fit(train_2,train['Sentiment'])
result = rfc.predict(test_2)
'''
from catboost import CatBoostClassifier
cbc = CatBoostClassifier(verbose=200,task_type='GPU')
cbc.fit(train_2,train['Sentiment'])
result = cbc.predict(test_2)

In [ ]:
sub = pd.read_csv('/kaggle/input/sentiment-analysis-on-movie-reviews/sampleSubmission.csv')
sub['Sentiment'] = result
sub.to_csv('result.csv',index=False)